In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as mtick
plt.style.use('ggplot')
import seaborn as sns
pd.options.display.float_format = '{:,.0f}'.format
import dataframe_image as dfi
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
lm = LinearRegression()
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
encoder = OneHotEncoder(sparse_output=False)
from sklearn.exceptions import ConvergenceWarning
import warnings

In [2]:
#Import Imputeed Ames Data
Ames = pd.read_csv('Ames_HousePrice_Imputed.csv')

#Update with new Ames Data
Ames_Columns_Numeric = Ames[[
    '1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath',
    'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'KitchenAbvGr',
    'LotArea', 'LotFrontage', 'LowQualFinSF', 'MasVnrArea', 'MiscVal', 'MoSold',
    'OpenPorchSF', 'PoolArea', 'ScreenPorch', 'TotalBsmtSF', 'TotRmsAbvGrd', 'WoodDeckSF',
    'YearBuilt', 'YearRemodAdd', 'YrSold'
]]
Ames_Columns_Ordinal = Ames[[ 
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual',
    'Electrical', 'ExterCond', 'ExterQual', 'Fence', 'FireplaceQu', 'Functional',
    'GarageCond', 'GarageFinish', 'GarageQual', 'HeatingQC', 'KitchenQual',
    'LandSlope', 'LotShape', 'OverallCond', 'OverallQual', 'PavedDrive',
    'PoolQC', 'Utilities'
]]
Ames_Columns_Nominal = Ames[[ 
    'Alley', 'BldgType', 'CentralAir', 'Condition1', 'Condition2',
    'Exterior1st', 'Exterior2nd', 'Foundation', 'GarageType', 'Heating',
    'HouseStyle', 'LandContour', 'LotConfig', 'MasVnrType', 'MiscFeature',
    'MSSubClass', 'MSZoning', 'Neighborhood', 'RoofMatl', 'RoofStyle',
    'SaleCondition', 'SaleType', 'Street'
]].astype(str)

#Combination Table of Numeric & Ordinal Columns
Ames_Columns_Numeric_Ordinal = Ames[
    list(Ames_Columns_Numeric.columns) + list(Ames_Columns_Ordinal.columns)
]
#Combination Table of Nominal & Ordinal Columns
Ames_Columns_Nominal_Ordinal = Ames[
    list(Ames_Columns_Nominal.columns) + list(Ames_Columns_Ordinal.columns)
]

In [3]:
Y = Ames['SalePrice']
lm = LinearRegression()
encoder = OneHotEncoder(sparse_output=False)

In [4]:
encoder_nominal = OneHotEncoder(sparse_output=False)
encoder_ordinal = OneHotEncoder(sparse_output=False)
Nominal_cols = Ames_Columns_Nominal.columns
Ordinal_cols = Ames_Columns_Ordinal.columns
Numeric_cols = Ames_Columns_Numeric.columns

# --- One-Hot Encode Nominal ---
Ames_Optimal_Nominal_One_Hot = encoder_nominal.fit_transform(Ames[Nominal_cols])
Ames_Optimal_Nominal_One_Hot_df = pd.DataFrame(
    Ames_Optimal_Nominal_One_Hot,
    columns=encoder_nominal.get_feature_names_out(Nominal_cols),
    index=Ames.index
)
#One-Hot Encode Ordinal
Ames_Optimal_Ordinal_One_Hot = encoder_ordinal.fit_transform(Ames[Ordinal_cols])
Ames_Optimal_Ordinal_One_Hot_df = pd.DataFrame(
    Ames_Optimal_Ordinal_One_Hot,
    columns=encoder_ordinal.get_feature_names_out(Ordinal_cols),
    index=Ames.index
)
#Merge Nominal & Ordinal Encodings
Ames_Optimal_Categorical_One_Hot = pd.merge(
    Ames_Optimal_Nominal_One_Hot_df,
    Ames_Optimal_Ordinal_One_Hot_df,
    left_index=True,
    right_index=True,
    how='left'
)
#Merge with Numeric Features
Ames_MLR_Optimal_Columns = pd.merge(
    Ames[Numeric_cols],
    Ames_Optimal_Categorical_One_Hot,
    left_index=True,
    right_index=True,
    how='left'
)

In [5]:
scaler_Y = StandardScaler()
scaler_X = StandardScaler()
Y_scaled = Y  #scaler_Y.fit_transform(Y.values.reshape(-1, 1)).flatten()
X_numeric_scaled = pd.DataFrame(
    scaler_X.fit_transform(Ames[Numeric_cols]),
    columns=Numeric_cols,
    index=Ames.index
)
X_scaled = pd.concat([X_numeric_scaled, Ames_Optimal_Categorical_One_Hot], axis=1)

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=2)
pipelines = {
    'Ridge': Pipeline(steps=[('regressor', Ridge())]),
    'Lasso': Pipeline(steps=[('regressor', Lasso(max_iter=100))]),
    'ElasticNet': Pipeline(steps=[('regressor', ElasticNet(max_iter=100))])
}
cv_results = {}
for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_scaled, Y_scaled, cv=kf, scoring='r2', n_jobs=-1)
    cv_results[name] = {
        'mean_R2': float(round(scores.mean(), 4)),
        'fold_R2': [float(round(s, 4)) for s in scores]
    }
print('Pipeline Cross_Val_Score Results (alpha=1)')
cv_results

Pipeline Cross_Val_Score Results (alpha=1)


{'Ridge': {'mean_R2': 0.9089,
  'fold_R2': [0.9114, 0.9338, 0.8411, 0.9265, 0.9315]},
 'Lasso': {'mean_R2': 0.9042,
  'fold_R2': [0.9, 0.9348, 0.8393, 0.9227, 0.9241]},
 'ElasticNet': {'mean_R2': 0.8532,
  'fold_R2': [0.8647, 0.8586, 0.8114, 0.855, 0.8766]}}

In [7]:
#Ridge Regression- Optimal Alpha via GridSearchCV
''' NOT UPDATED
ridge = Ridge()
alpha_ridge_grid = np.linspace(1, 20, 100)

# Setup Grid Search for Ridge
ridge_grid = GridSearchCV(estimator=pipelines['Ridge'],
                          param_grid={'regressor__alpha': alpha_ridge_grid},
                          verbose=1)
ridge_grid.fit(X_scaled, Y_scaled)
 
#Best alpha & score
print('\nRidge Pipeline GridSearchCV Results\n')
ridge_best_alpha = ridge_grid.best_params_['regressor__alpha']
ridge_best_score = ridge_grid.best_score_
print(f"Best Alpha for Ridge: {ridge_best_alpha}")
print(f"Best CV Score: {round(ridge_best_score, 4)}")
'''

Fitting 5 folds for each of 100 candidates, totalling 500 fits

Ridge Pipeline GridSearchCV Results

Best Alpha for Ridge: 2.727272727272727
Best CV Score: 0.9167


In [37]:
#Ridge Regression- Optimal Alpha via RidgeCV
''' NOT UPDATED
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

ridge_cv = RidgeCV(alphas=np.linspace(1, 20, 100), cv=kf, scoring='r2')
ridge_cv.fit(X_scaled, Y_scaled)

print('RidgeCV Results\n')
alpha_opt_ridge = ridge_cv.alpha_
print(f"Optimal Ridge Alpha: {alpha_opt_ridge:.4f}")

#Fit Ridge with optimal alpha
ridge.set_params(alpha=alpha_opt_ridge)
ridge.fit(X_scaled, Y_scaled)
# Coefficients as a Series with feature names
ridge_coefs = pd.Series(ridge.coef_, index=X_scaled.columns)
ridge.score(X_scaled, Y_scaled)

#Scoring & Coefficients
cv_score_ridge = cross_val_score(ridge, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"Mean R²: {cv_score_ridge.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_score_ridge])}")

ridge_coefs_formatted = ridge_coefs.sort_values(key=abs, ascending=False)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
print("\nTop 20 largest coefficients (by absolute value):")
print(ridge_coefs_formatted.head(20))

#Zero Coefficients
zero_coefs_ridge = ridge_coefs[ridge_coefs == 0].index.tolist()
print(f"\nNumber of features with zero coefficient: {len(zero_coefs_ridge)}")
print("Zero coefficient features:")
print(zero_coefs_ridge)
'''

Optimal Ridge Alpha: 5.7980

Mean R²: 0.9118;   Fold R² scores: 0.9182, 0.9334, 0.8430, 0.9276, 0.9368
Top 20 largest coefficients (by absolute value):
OverallQual_9            44505.6101
Condition2_PosN         -40556.3540
OverallQual_8            28777.3754
Neighborhood_GrnHill     26613.0678
RoofMatl_WdShngl         26300.7540
Neighborhood_StoneBr     26074.4905
Neighborhood_NoRidge     20224.5891
Condition2_PosA          20089.1284
OverallCond_8            18517.6847
OverallCond_2           -18473.2041
PoolQC_4                 18195.2419
ExterQual_3              16717.1650
OverallQual_3           -15520.9404
OverallCond_1           -15511.5074
GrLivArea                15492.8645
Exterior1st_BrkFace      15490.0801
BldgType_Twnhs          -14876.2795
SaleCondition_Partial    14640.4181
Neighborhood_Edwards    -14191.3645
OverallCond_7            14110.8553
dtype: float64

Number of features with zero coefficient: 0
Zero coefficient features:
[]


In [9]:
#Lasso Regression- Optimal Alpha via GridSearchCV
lasso = Lasso()
alpha_lasso_grid = np.linspace(29, 31, 100)
 
# Setup Grid Search for Ridge
lasso_grid = GridSearchCV(estimator=pipelines['Lasso'],
                          param_grid={'regressor__alpha': alpha_lasso_grid},
                          verbose=1)
lasso_grid.fit(X_scaled, Y_scaled)
print('\nLasso Pipeline GridSearchCV Results\n')

#Best alpha & score
lasso_best_alpha = lasso_grid.best_params_['regressor__alpha']
lasso_best_score = lasso_grid.best_score_
print(f"Best Alpha for lasso: {lasso_best_alpha:.6f}")
print(f"Best CV Score {round(lasso_best_score, 4)}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits

Lasso Pipeline GridSearchCV Results

Best Alpha for lasso: 30.333333
Best CV Score 0.9168


In [10]:
#Lasso Regression- Optimal Alpha via LassoCV
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

lasso_cv = LassoCV(alphas=np.linspace(0, 100, 100), cv=kf, n_jobs=-1, max_iter=5000)
lasso_cv.fit(X_scaled, Y_scaled)

print('LassoCV Results\n')
alpha_opt_lasso = lasso_cv.alpha_
print(f"Optimal Lasso Alpha: {alpha_opt_lasso:.4f}")

#Fit Lasso with optimal alpha
lasso = Lasso(alpha=alpha_opt_lasso, max_iter=5000)
lasso.fit(X_scaled, Y_scaled)

lasso_coefs = pd.Series(lasso.coef_, index=X_scaled.columns)
lasso_coefs_formatted = lasso_coefs.sort_values(key=abs, ascending=False)
pd.set_option('display.float_format', lambda x: f'{x:.10f}')

#Scores & Coefficients
cv_scores_lasso = cross_val_score(lasso, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"\nMean R² Scores: {cv_scores_lasso.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_scores_lasso])}")

print("\nTop 20 largest coefficients:")
print(lasso_coefs_formatted.head(20))

#Zero Coefficients
zero_coefs_lasso = lasso_coefs[lasso_coefs == 0].index.tolist()
non_zero_coefs_lasso = lasso_coefs[~lasso_coefs.index.isin(zero_coefs_lasso)]
print(f"\nNumber of features with Non-Zero coefficient: {len(non_zero_coefs_lasso)}")
print(f"\nNumber of features with Zero coefficient: {len(zero_coefs_lasso)}")
print("Zero coefficient features:")
print(zero_coefs_lasso)

LassoCV Results

Optimal Lasso Alpha: 30.3030

Mean R² Scores: 0.9095;   Fold R² scores: 0.9098, 0.9366, 0.8359, 0.9324, 0.9330

Top 20 largest coefficients:
Condition2_PosN         -113339.4406687605
OverallQual_9             76662.7129700939
Neighborhood_GrnHill      76153.5235557262
OverallQual_8             49852.1720930153
Neighborhood_StoneBr      32765.6261547613
RoofMatl_WdShngl          32250.6673367235
PoolQC_4                  30811.9846227997
Neighborhood_NoRidge      26920.3302886245
GrLivArea                 25496.6024866434
OverallQual_7             21078.3184342566
OverallCond_8             19738.2420736662
OverallCond_1            -19221.4209694122
OverallCond_2            -18969.0495507083
Neighborhood_Somerst      18066.6991253673
ExterQual_3               17596.5665164133
Neighborhood_NridgHt      16721.9840356886
SaleCondition_Family     -16415.7468771220
Neighborhood_Crawfor      15546.8084605966
SaleCondition_Partial     15140.6178237595
OverallCond_7            

In [35]:
#ElasticNet Regression- Optimal Alpha via GridSearchCV
''' NOT UPDATED
l1_ratio = np.linspace(0.1, 1, 20)
alpha_elastic_grid = np.linspace(25, 35, 100)

#Setup Grid Search for Ridge
elasticnet_grid = GridSearchCV(estimator=pipelines['ElasticNet'],
                               param_grid={'regressor__alpha': alpha_elastic_grid,
                                           'regressor__l1_ratio': l1_ratio},
                               verbose=1)
elasticnet_grid.fit(X_scaled, Y_scaled)


print('\nElasticNet Pipeline GridSearchCV Results\n')

#Best alpha & score
elasticnet_best_rho = elasticnet_grid.best_params_['regressor__l1_ratio']
elasticnet_best_alpha = elasticnet_grid.best_params_['regressor__alpha']
elasticnet_best_score = elasticnet_grid.best_score_
print(f"Best Rho for ElasticNet: {elasticnet_best_rho:.4f}")
print(f"Best Alpha for ElasticNet: {elasticnet_best_alpha:.6f}")
print(f"Best CV Score {round(elasticnet_best_score, 4)}")
'''

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits
Best Rho for ElasticNet: 1.0000
Best Alpha for ElasticNet: 29.949495
Best CV Score 0.9173


In [28]:
#ElasticNet Regression- Optimal Alpha via ElasticNetCV
''' NOT UPDATED
elastic_cv = ElasticNetCV(
    l1_ratio=np.linspace(.01, 1, 100),
    alphas=np.linspace(.0001, 100, 100),
    cv=5, max_iter=5000, n_jobs=-1
)
elastic_cv.fit(X_scaled, Y_scaled)

print('ElasticNetCV Results\n')
best_alpha_elastic = elastic_cv.alpha_
best_l1_elastic = elastic_cv.l1_ratio_
print(f"Optimal ElasticNet l1_ratio: {best_l1_elastic:.2f}")
print(f"Optimal ElasticNet Alpha: {best_alpha_elastic:.4f}")


#Build final ElasticNet model using best parameters
final_en = ElasticNet(alpha=best_alpha_elastic, l1_ratio=best_l1_elastic, max_iter=10000)
final_en.fit(X_scaled, Y_scaled)

#Scires
cv_scores_elasticnet = cross_val_score(final_en, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"Mean CV R²: {cv_scores_elasticnet.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_scores_elasticnet])}\n")

#Coefficient Series
coef_elastic = pd.Series(final_en.coef_, index=X_scaled.columns)
zero_features_elastic = coef_elastic[coef_elastic == 0].index.tolist()
nonzero_features_elastic = coef_elastic[coef_elastic != 0]

top20_elastic = nonzero_features_elastic.reindex(nonzero_features_elastic.abs().sort_values(ascending=False).index).head(20)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
print("Top 20 largest non-zero coefficients (abs, 4 decimals):")
print(top20_elastic)

#Zero Coefficients
print(f"\nNumber of features with Non-Zero coefficient: {len(nonzero_features_elastic)}")
print(f"\nNumber of features with Zero coefficient: {len(zero_features_elastic)}")
print("Zero coefficient features:")
print(zero_features_elastic)
'''

Optimal ElasticNet Alpha: 27.2728
Optimal ElasticNet l1_ratio: 1.00
Mean CV R²: 0.9097;   Fold R² scores: 0.9098, 0.9370, 0.8371, 0.9322, 0.9323

Top 20 largest non-zero coefficients (abs, 4 decimals):
Condition2_PosN         -116121.8525
Neighborhood_GrnHill      79934.0563
OverallQual_9             76557.8801
OverallQual_8             50067.2800
RoofMatl_WdShngl          37643.9799
PoolQC_4                  35484.6816
Neighborhood_StoneBr      33293.4158
Neighborhood_NoRidge      26924.5370
GrLivArea                 25573.2735
SaleCondition_Partial     21515.9909
OverallQual_7             21233.5132
OverallCond_1            -20586.7423
OverallCond_8             20395.9223
OverallCond_2            -18392.9263
Neighborhood_Somerst      18354.5088
ExterQual_3               17507.4366
Neighborhood_NridgHt      16779.7222
BldgType_Twnhs           -16470.8351
OverallCond_7             15295.4688
Neighborhood_Crawfor      15079.6312
dtype: float64

Number of features with Non-Zero coefficie

In [34]:
#ElasticNet.5 Regression where Rho = .5- Optimal Alpha via GridSearchCV
''' NOT UPDATED
l1_ratio_5 = [.5]
alpha_elastic5_grid = np.linspace(.002, .003, 100)
 
#Setup Grid Search for Ridge
elasticnet5_grid = GridSearchCV(estimator=pipelines['ElasticNet'],
                               param_grid={'regressor__alpha': alpha_elastic5_grid,
                                           'regressor__l1_ratio': l1_ratio_5},
                               verbose=1)
elasticnet5_grid.fit(X_scaled, Y_scaled)

print('\nElasticNet Pipeline GridSearchCV Results (Rho = .5)\n')

#Best alpha & score
elasticnet5_best_rho = elasticnet5_grid.best_params_['regressor__l1_ratio']
elasticnet5_best_alpha = elasticnet5_grid.best_params_['regressor__alpha']
elasticnet5_best_score = elasticnet5_grid.best_score_
print(f"Best Rho for ElasticNet: {elasticnet5_best_rho:.4f}")
print(f"Best Alpha for ElasticNet: {elasticnet5_best_alpha:.6f}")
print(f"Best CV Score {round(elasticnet5_best_score, 4)}")
'''

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Rho for ElasticNet: 0.5000
Best Alpha for ElasticNet: 0.002535
Best CV Score 0.9169


In [33]:
#ElasticNetCV where Rho = .5
''' NOT UPDATED
elastic5_cv = ElasticNetCV(
    l1_ratio=.5,
    alphas=np.linspace(.0023, .0025, 100),
    cv=5,
    max_iter=5000,
    n_jobs=-1
)
elastic5_cv.fit(X_scaled, Y_scaled)

print('ElasticNetCV Results (Rho = .5)\n')
best_alpha_elastic5 = elastic5_cv.alpha_
best_l1_elastic5 = elastic5_cv.l1_ratio_
print(f"Optimal ElasticNet.5 l1_ratio: {best_l1_elastic5:.2f}")
print(f"Optimal ElasticNet.5 Alpha: {best_alpha_elastic5:.4f}")

#Build final ElasticNet model using best parameters
final_en = ElasticNet(alpha=best_alpha_elastic5, l1_ratio=best_l1_elastic5, max_iter=10000)
final_en.fit(X_scaled, Y_scaled)  #Fit final model

#Scores
cv_scores_elasticnet5 = cross_val_score(final_en, X_scaled, Y_scaled, cv=kf, scoring='r2')
print(f"Mean CV R²: {cv_scores_elasticnet5.mean():.4f};  ", f"Fold R² scores: {', '.join([f'{s:.4f}' for s in cv_scores_elasticnet5])}\n")

#Coefficients as a Series
coef_elastic5 = pd.Series(final_en.coef_, index=X_scaled.columns)

#Summary of coefficients
zero_features_elastic5 = coef_elastic5[coef_elastic5 == 0].index.tolist()
nonzero_features_elastic5 = coef_elastic5[coef_elastic5 != 0]

top20_elastic5 = nonzero_features_elastic5.reindex(nonzero_features_elastic5.abs().sort_values(ascending=False).index).head(20)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
print("Top 20 largest non-zero coefficients (abs, 4 decimals):")
print(top20_elastic5)

#Zero coefficients
print(f"\nNumber of features with Non-Zero coefficient: {len(nonzero_features_elastic5)}")
print(f"\nNumber of features with Zero coefficient: {len(zero_features_elastic5)}")
print("Zero coefficient features:")
print(zero_features_elastic5)
'''

Optimal ElasticNet.5 Alpha: 0.0024
Optimal ElasticNet.5 l1_ratio: 0.50
Mean CV R²: 0.9111;   Fold R² scores: 0.9161, 0.9340, 0.8421, 0.9279, 0.9352

Top 20 largest non-zero coefficients (abs, 4 decimals):
Condition2_PosN         -59684.6885
OverallQual_9            51349.8627
Neighborhood_GrnHill     40827.1073
RoofMatl_WdShngl         32455.3003
OverallQual_8            32237.0099
Neighborhood_StoneBr     28131.0697
PoolQC_4                 27710.2629
Condition2_PosA          27620.1428
OverallCond_8            21067.0341
Neighborhood_NoRidge     20664.1056
OverallCond_2           -18970.3668
OverallCond_1           -18897.2197
SaleCondition_Partial    17323.9848
Exterior1st_BrkFace      16306.5416
OverallQual_3           -16244.0213
OverallCond_7            15859.9200
ExterQual_3              15712.8645
GarageQual_5             15650.2269
BldgType_Twnhs          -15621.2552
GrLivArea                15461.3850
dtype: float64

Number of features with Non-Zero coefficient: 296

Number o

In [42]:
#Set optimal hyperparameters
''' NOT UPDATED
pipelines['Ridge'].set_params(regressor__alpha=ridge_best_alpha)
pipelines['Lasso'].set_params(regressor__alpha=lasso_best_alpha)
pipelines['ElasticNet'].set_params(regressor__alpha=elasticnet5_best_alpha, regressor__l1_ratio=0.5)

#Cross-validation
cv_results = {}
for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_scaled, Y_scaled, cv=kf, scoring='r2', n_jobs=-1)
    cv_results[name] = {
        'mean_R2': float(round(scores.mean(), 4)),
        'fold_R2': [float(round(s, 4)) for s in scores]
    }
print('Pipeline Cross_Val_Score Results (Optimal Alpha)')
cv_results
'''

{'Lasso': {'mean_R2': 0.9097,
  'fold_R2': [0.9102, 0.9369, 0.8363, 0.9321, 0.9328]},
 'Ridge': {'mean_R2': 0.9111,
  'fold_R2': [0.9162, 0.934, 0.8422, 0.9279, 0.9352]},
 'ElasticNet': {'mean_R2': 0.9112,
  'fold_R2': [0.9163, 0.934, 0.8422, 0.928, 0.9353]}}